# Satellite SR Training on Google Colab T4

**Goal**: Train SwinIR with full settings (batch_size=8, patch_size=128×128)

**Expected time**: 4-5 hours for 100 epochs

## Step 1: Check GPU

In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ NO GPU! Go to Runtime → Change runtime type → T4 GPU")

## Step 2: Clone Your GitHub Repo

In [ ]:
# Clone repo
!git clone https://github.com/Elkrisent/klymo-ascent-1.0.git
%cd klymo-ascent-1.0

## Step 3: Install Dependencies

In [ ]:
!pip install -r requirements.txt
!pip install rasterio scikit-image scikit-learn

## Step 4: Get the Dataset

**Option A: Download from Google Drive (if Ragunath shared it)**

According to your README, the dataset is on Google Drive. Get the link from README.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# If dataset is in Drive at: /content/drive/MyDrive/satellite-sr-data/
# Create symlink
!ln -s /content/drive/MyDrive/satellite-sr-data/worldstrat_processed worldstrat_processed

**Option B: Download directly (if Ragunath provided a download link)**

In [ ]:
# Example: if there's a direct download link
# !wget <link-to-worldstrat_processed.zip>
# !unzip worldstrat_processed.zip

## Step 5: Clone SwinIR Repository

In [ ]:
# Clone SwinIR repo
!git clone https://github.com/JingyunLiang/SwinIR.git

# Download pretrained weights
!mkdir -p SwinIR/model_zoo
!wget -P SwinIR/model_zoo https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/001_classicalSR_DIV2K_s48w8_SwinIR-M_x4.pth

## Step 6: Verify Data Pipeline

In [ ]:
from dataset import SatelliteSRDataset
from torch.utils.data import DataLoader

dataset = SatelliteSRDataset(
    hr_dir='worldstrat_processed/HR_8bit',
    lr_patch_size=128,
    scale=4
)

loader = DataLoader(dataset, batch_size=4, shuffle=True)
lr, hr = next(iter(loader))

print(f"✅ LR shape: {lr.shape}")  # expect [4, 3, 128, 128]
print(f"✅ HR shape: {hr.shape}")  # expect [4, 3, 512, 512]
print(f"✅ LR range: {lr.min():.3f} → {lr.max():.3f}")
print(f"✅ HR range: {hr.min():.3f} → {hr.max():.3f}")

## Step 7: Run Training

This will take 4-5 hours. The notebook will auto-checkpoint every 20 epochs.

In [ ]:
# Run training
!python train_swinir.py

## Step 8: Monitor Training (Optional)

Run this in a separate cell while training:

In [ ]:
# Watch GPU usage
!nvidia-smi

## Step 9: Download Checkpoint

After training completes:

In [ ]:
from google.colab import files

# Download the best checkpoint
files.download('checkpoints/swinir_best.pth')

## Step 10: Evaluate Results

In [ ]:
# Run bicubic baseline
!python eval_bicubic.py

In [ ]:
# Check final training metrics
import torch

ckpt = torch.load('checkpoints/swinir_best.pth')
print(f"Best PSNR: {ckpt['psnr']:.2f} dB")
print(f"Final Epoch: {ckpt['epoch']}")

## Expected Results

With full settings on T4 GPU:

```
Bicubic:  ~30 dB
SwinIR:   33-35 dB
Improvement: +3-5 dB
```

This is what judges want to see!